# Detectability Map
@author: Max Felius

Detectability map using a 1x1 km subset of track 88 Sentinel-1 data.

The goal is to test the subset on every position to determine if it is possible to detect a particular sinkhole (retrieved from digitised sinkhole analysis) if it would happen at that particular position. 

In [1]:
#imports
import numpy as np
import pandas as pd
import sys, os, time
import matplotlib.pyplot as plt
from tqdm import tqdm

#import self created package
# from Detectability_Map.Detectability_Map_Creator import detectability_map
from Detectability_Map.read_sentinel1_csv_data import dataset
from Detectability_Map.rijksdriehoek import Rijksdriehoek

In [2]:
folder = 'Subsets'
filename = 'subset_r400_lon6.06_lat50.87.csv'
dataset_filename = os.path.join(folder,filename)

dataset_obj = dataset(dataset_filename)

extend_rd = dataset_obj.extend_rd #[min x, max x, min y, max y]
extend_wgs = dataset_obj.extend_wgs #[min lon, max lon, min lat, max lat]

Starting to read the data: Subsets/subset_r400_lon6.06_lat50.87.csv.
Finished Loading the dataset in 0.07316470146179199 seconds...


In [3]:
#defining the different influence functions
def zg(R,r,itype='gaus'):
    # Automatically uses the Gaussian Influence Function unless specified differently
    if itype == 'gaus':
        return -zg_gaus(R,r) 
    elif itype == 'bals':
        return -zg_bals(R,r)    
    elif itype == 'beyer':
        return -zg_beyer(R,r)    
    else:
        print(f'Unknown itype: {itype}.')

def zg_gaus(R,r):
    return (1/(R*R))*np.exp(-np.pi*(r**2/R**2))

def zg_bals(H,r):
    r = np.sqrt((x-x0)**2 + (y-y0)**2)
    zone = np.arctan(r/H)
    return np.cos(zone)**2

def zg_beyer(R,r):
#     r = np.sqrt((x-x0)**2 + (y-y0)**2)\n",
    kz = ((3)/(np.pi*R**2))*(1-(r/R)**2)**2
    kz[r>R] = 0
    return kz

In [4]:
def check_subset(subset,R,x0,y0):
    '''
    Checks if a subset has a solvable design matrix

    Input:
    :type subset: pandas dataframe
    :type S: int
    :type R: int
    :type x0: int
    :type y0: int

    Output
    :rtype: boolean, float
    '''
    #sinkhole parameters
    v = 50
#     t = [0,12,24,36]
    t = 12
    
    itype = 'gaus'
    
    # print('Computing the radius.')
    r = np.sqrt((subset['pnt_rdx'].values-x0)**2 + (subset['pnt_rdy'].values-y0)**2)

    if len(r)==0:
        return 0, 0.0
    else:
        #defining the jacobian matrix for nonlinear least squares
        A1 = t*zg(R,r,itype)
        A2 = ((2*R**2 + 2*np.pi*r**2)/(R**3))*(v*t)*zg(R,r,itype)
        J = np.array([A1,A2]).T
        cond_number = np.linalg.cond(J)

    if cond_number > 1/sys.float_info.epsilon:
        return 0, cond_number
    else:
        return 1, cond_number

In [13]:
def make_map(R_list,x_eval,y_eval,nx,ny,dataset_obj):
    '''
    Method to create the detectability map. Input are the evaluation coordinates and the radius of infuence.

    Input:_pos
    :type R_list: list[int]
    :type S: int
    :type x_eval: list[float], rd
    :type y_eval: list[float], rd
    :type nx: int
    :type ny: int
    :type dataset_obj: object(dataset)

    Output:
    :rtype pandas.DataFrame
    '''
    header_outputframe = ['pnt_lon','pnt_lat','pnt_rdx','pnt_rdy','geometry (wgs)','geometry (rd)','sum']
    
    #implement R columns in the header
    for item in R_list:
        header_outputframe.append(f'R_{item}')
        header_outputframe.append(f'Cond_R_{item}')
    
    # building the numbers for in the dataframe
    lon_eval_wgs, lat_eval_wgs = Rijksdriehoek(x_eval, y_eval).to_wgs()
    fill_data = np.zeros((len(lon_eval_wgs),len(header_outputframe)-4))
    coordinate_data = np.array([lon_eval_wgs,lat_eval_wgs,x_eval,y_eval]).T
    coordinate_data = np.concatenate((coordinate_data,fill_data),axis=1)
    
    # Building dataframe
    df = pd.DataFrame(coordinate_data,columns=header_outputframe)
    
    for idx,R in enumerate(R_list):
        print(f'Set {idx+1}/{len(R_list)}.')
        for i in tqdm(range(len(x_eval)),desc='Making the Map...'):
            #select center position
            x_pos = x_eval[i]
            y_pos = y_eval[i]

            #create the subset
            subset = dataset_obj.create_spatial_subset([x_pos,y_pos],R)

            result, cond = check_subset(subset,R,x_pos,y_pos)

            #filling the dataframe
            #create a polygon of the entry
            lbrd = Rijksdriehoek(x_pos-min(R_list)/2,y_pos-min(R_list)/2)
            rbrd = Rijksdriehoek(x_pos+min(R_list)/2,y_pos-min(R_list)/2)
            rtrd = Rijksdriehoek(x_pos+min(R_list)/2,y_pos+min(R_list)/2)
            ltrd = Rijksdriehoek(x_pos-min(R_list)/2,y_pos+min(R_list)/2)

            leftbot = lbrd.to_wgs()
            rightbot = rbrd.to_wgs()
            righttop = rtrd.to_wgs()
            lefttop = ltrd.to_wgs()

            df['geometry (wgs)'].iloc[i] = f'POLYGON (({leftbot[1]} {leftbot[0]},{rightbot[1]} {rightbot[0]},{righttop[1]} {righttop[0]},{lefttop[1]} {lefttop[0]}))'
            df['geometry (rd)'].iloc[i] = f'POLYGON (({lbrd.rd_x} {lbrd.rd_y},{rbrd.rd_x} {rbrd.rd_y},{rtrd.rd_x} {rtrd.rd_y},{ltrd.rd_x} {ltrd.rd_y}))'

            df[f'R_{R}'].iloc[i] = result
            df[f'Cond_R_{R}'].iloc[i] = cond
            
            if result == 1:
                df['sum'].iloc[i] += 1
        
    return df

In [16]:
# #parameters
# n = 100
# w = 1
# M = 1
# H = 10
# zone_angle = np.deg2rad(35)
# w_c = H/np.cos(zone_angle)

# S = (2*M*w)/(w+w_c)
# R = 10 #[m]

R_list = [10,20,30,40,50,60,70,80]

coordinates = extend_rd
xmin = coordinates[0]
xmax = coordinates[1]

x_range = np.arange(xmin,xmax,min(R_list)/2)

ymin = coordinates[2]
ymax = coordinates[3]

y_range = np.arange(ymin,ymax,min(R_list)/2)

xv,yv = np.meshgrid(x_range,y_range)

x_eval = xv.ravel()
y_eval = yv.ravel()


df = make_map(R_list,x_eval,y_eval,len(x_range),len(y_range),dataset_obj)

Making the Map...:   0%|          | 0/66010 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
Making the Map...:   0%|          | 43/66010 [00:00<07:03, 155.74it/s]

Set 1/8.


Making the Map...:   0%|          | 18/66010 [00:00<06:36, 166.64it/s]

Set 2/8.


Making the Map...:   0%|          | 34/66010 [00:00<06:20, 173.22it/s]

...

Making the Map...:   0%|          | 15/66010 [00:00<07:26, 147.84it/s]

Set 7/8.


Making the Map...:   0%|          | 18/66010 [00:00<06:09, 178.50it/s]

Set 8/8.


Making the Map...: 100%|██████████| 66010/66010 [07:53<00:00, 139.48it/s]


In [17]:
df.to_csv('test1.csv')

In [15]:
df

,pnt_lon,pnt_lat,pnt_rdx,pnt_rdy,geometry (wgs),geometry (rd),sum,R_30,Cond_R_30
0,50.861210,6.051469,201770.36798,319255.161639,POLYGON ((6.051254238253494 50.861076039414655...,POLYGON ((201755.36798023578 319240.1616393676...,1.0,1.0,1.000000
1,50.861208,6.051682,201785.36798,319255.161639,"POLYGON ((6.051467265271871 50.86107481574402,...",POLYGON ((201770.36798023578 319240.1616393676...,1.0,1.0,96.668101
2,50.861207,6.051895,201800.36798,319255.161639,"POLYGON ((6.051680292277208 50.861073591681,6....",POLYGON ((201785.36798023578 319240.1616393676...,0.0,0.0,0.000000
3,50.861206,6.052108,201815.36798,319255.161639,POLYGON ((6.051893319269499 50.861072367225624...,POLYGON ((201800.36798023578 319240.1616393676...,0.0,0.0,0.000000
4,50.861205,6.052321,201830.36798,319255.161639,"POLYGON ((6.052106346248739 50.8610711423779,6...",POLYGON ((201815.36798023578 319240.1616393676...,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...
7447,50.875557,6.065314,202730.36798,320860.161639,POLYGON ((6.065099106068937 50.875423105981056...,POLYGON ((202715.36798023578 320845.1616393676...,1.0,1.0,17.394697
7448,50.875555,6.065527,202745.36798,320860.161639,"POLYGON ((6.065312198596273 50.87542185668645,...",POLYGON ((202730.36798023578 320845.1616393676...,1.0,1.0,17.269553
7449,50.875554,6.065740,202760.36798,320860.161639,POLYGON ((6.065525291110288 50.875420606999334...,POLYGON ((202745.36798023578 320845.1616393676...,1.0,1.0,16.579560
7450,50.875553,6.065953,202775.36798,320860.161639,POLYGON ((6.0657383836109755 50.87541935691969...,POLYGON ((202760.36798023578 320845.1616393676...,1.0,1.0,27.166108
